In [12]:
import time
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [3]:
model  = tf.keras.models.load_model("C150.h5")

In [14]:
df = pd.read_csv(r"CNNdata\fulldata.csv")
df["DateTime"] = pd.to_datetime(df["DateTime"])
df.set_index(df["DateTime"], inplace=True)
df = df.resample("50ms").mean()
df = df.fillna(method='ffill') 
del df["DateTime"]


C:\Users\dbjac\AppData\Local\Temp\ipykernel_17800\2532512776.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [8]:
def create_target(df, prediction_horizon=10):
    target = np.zeros(len(df) - prediction_horizon)  
    for i in range(len(df) - prediction_horizon):
      current_price = df['Mid'].iloc[i]
      future_price = df['Mid'].iloc[i + prediction_horizon]

      if future_price > current_price:
          target[i] = 2  
      elif future_price < current_price:
          target[i] = 0
      else:
          target[i] = 1  

    return target

def create_image_data(X,time_window):
    num_samples = len(X) - time_window + 1
    num_channels = X.shape[1]
    images = np.zeros((num_samples,num_channels,time_window))
    for i in range(num_samples):
        images[i] = X[i:i+time_window].T
    return images

In [15]:

time_window = 15
prediction_horizon =10
X = df.drop(["Next_Mid_Direction", "Mid_Change"], axis=1)  
X_train = X[:-prediction_horizon]

y = create_target(df, prediction_horizon)  

X_train["Size"] = X_train["Size"].astype(np.float64)


numerical_cols = X_train.select_dtypes(include=np.float64).columns
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

X_train_images = create_image_data(X_train, time_window)



C:\Users\dbjac\AppData\Local\Temp\ipykernel_17800\4290216542.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["Size"] = X_train["Size"].astype(np.float64)
C:\Users\dbjac\AppData\Local\Temp\ipykernel_17800\4290216542.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])


In [21]:

batch_size = 32

inference_times = []
num_samples = len(X_train_images)
print(num_samples)
num_batches = int(np.ceil(num_samples / batch_size))  


inference_times = []
for i in range(num_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, num_samples)  
    batch = X_train_images[start_index:end_index]    
    start_time = time.time()
    _ = model.predict(batch,verbose=0)
    end_time = time.time()
    inference_times.append(end_time - start_time)

inference_time_per_sample_individual = np.mean(inference_times)
print(f"Average inference time per sample (individual timings): {inference_time_per_sample_individual:.6f} seconds")


467961
Average inference time per sample (individual timings): 0.065017 seconds
